In [1]:
import gdal
import matplotlib.pyplot as plt

In [7]:
ds = gdal.Open('LC80930862015040.tif')
ds_arr = ds.ReadAsArray()

In [3]:
srcband = ds.GetRasterBand(1)
band_arr = srcband.ReadAsArray()

In [4]:
plt.imshow(band_arr)
plt.show()

In [8]:
ds_arr.shape

(3, 178, 613)

In [ ]:
# read all band images in a directory

In [ ]:
# read all mask images in a directory

In [ ]:
# compose band and mask images into one image

In [ ]:
# use image names to look up metadata from csv export

In [ ]:
# compose all data into a single data frame

In [ ]:
# fill in metadata

In [ ]:
# ready for interpolation / training